<a href="https://colab.research.google.com/github/kashfay110/stock-market-python/blob/main/svr_ann_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
pip install mplfinance


In [36]:
# import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import pandas as pd
# from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import mplfinance as mpf
import keras
from keras.models import Sequential
from keras.layers import Dense

In [34]:
# Get the stock data
#df = quandl.get("WIKI/FB")
#df = pd.read_csv (r'dataset/FB.csv')
# Take a look at data
#print(df.head())

from google.colab import files
uploaded = files.upload()
import io
df = pd.read_csv(io.BytesIO(uploaded['FB.csv']))
df.head()

df2 = pd.read_csv(io.BytesIO(uploaded['FB.csv']))
#print(df2.head())
# Get the Adjusted Close
df = df[['Adj Close']]
# Take a look at new data
#print(df.head())

KeyboardInterrupt: ignored

In [ ]:
# A variable for predicting 'forecast_out' days out in future
forecast_out = 1

In [ ]:
# Create another column (dependent variable) shifted 'forecast_out' units up
df['Prediction'] = df[["Adj Close"]].shift(-forecast_out)
#print(df.tail())

In [ ]:
# Create the independent dataset
# Convert the dataframe to numpy array
X = np.array(df.drop(['Prediction'],1))
# Remove the last 'forecast_out' rows
X = X[:-forecast_out]
#print(X)

In [ ]:
# Create the dependent dataset
# Convert the dataframe to numpy array (All of the values including the NaNs)
y = np.array(df['Prediction'])
# Get all of the y values except the last 'forecast_out' rows
y = y[:-forecast_out]
#print(y)

In [ ]:
z = np.array(df2)
z = z[:-forecast_out]
print(z)

In [ ]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
# train_set = x_train + y_train
# test_set = x_test + y_test

In [ ]:
# Create and train the Support Vector Machine (Regressor)
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

In [ ]:
# Create and train the Linear Regression model
lr = LinearRegression()
# Train model
lr.fit(x_train, y_train)

In [ ]:
#Set x_forecast equal to the last forecast_out rows of the original dat set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
#print(x_forecast)

In [ ]:
# Print the SVR prediction for the next forecast_out days
svm_prediction = svr_rbf.predict(x_forecast)
print(f'The next {forecast_out} days by the SVR model: \n{svm_prediction}')

# Print the Linear Regression prediction for the next forecast_out days
lr_prediction = lr.predict(x_forecast)
print(f'The next {forecast_out} days by the LR model: \n{lr_prediction}')

In [ ]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction
# Best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svr confidence: ", svm_confidence)

# Testing Model: Score returns the coefficient of determination R^2 of the prediction
# It provides a measure of how well observed outcomes are replicated by the model, based on the proportion of total variation of outcomes explained by the model
# Best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

In [ ]:
## Creating predicted values for both models
y_pred_svr = svr_rbf.predict(x_test)
y_pred_lr = lr.predict(x_test)

In [ ]:
model = Sequential()
model.add(Dense(200, input_dim=1, activation='relu'))
model.add(Dense(200, input_dim=200, activation='relu'))
model.add(Dense(200, input_dim=200, activation='relu'))
model.add(Dense(1, activation='linear'))

In [ ]:
keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='mean_squared_error', optimizer='RMSprop', metrics=['mean_absolute_percentage_error'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train, epochs=20, batch_size=32,validation_split=0.15,validation_data=None,verbose=1)

In [ ]:
keras.backend.clear_session()